In [26]:
import pandas as pd
import numpy as np
from tqdm import tqdm

from sklearn.metrics.pairwise import cosine_similarity

view_log_train = pd.read_csv('view_log.csv')
article_info = pd.read_csv('article_info.csv')
submission = pd.read_csv('sample_submission.csv')

view_log_train

,userID,articleID,userRegion,userCountry
0,USER_0000,ARTICLE_0661,NY,US
1,USER_0000,ARTICLE_2316,NY,US
2,USER_0000,ARTICLE_1345,NY,US
3,USER_0000,ARTICLE_1089,NY,US
4,USER_0000,ARTICLE_1484,NY,US
...,...,...,...,...
42712,USER_1420,ARTICLE_0682,SP,BR
42713,USER_1420,ARTICLE_2179,SP,BR
42714,USER_1420,ARTICLE_1848,SP,BR
42715,USER_1420,ARTICLE_0030,SP,BR


In [30]:
remain = view_log_train.articleID.value_counts().keys()
remain = set(remain)

for i in tqdm(range(len(article_info))):
    if article_info.loc[i,'articleID'] not in remain:
        article_info = article_info.drop(i)
article_info

100%|███████████████████████████████████████████████████████████████████████████| 3008/3008 [00:00<00:00, 31017.44it/s]


,articleID,Title,Content,Format,Language,userID,userCountry,userRegion
0,ARTICLE_0000,19 Tips For Everyday Git Use,I've been using git full time for the past 4 y...,HTML,en,USER_0683,NaN,NaN
1,ARTICLE_0001,Intel buys computer vision startup Itseez to i...,Intel has acquired computer vision and machine...,HTML,en,USER_1129,NaN,NaN
2,ARTICLE_0002,Practical End-to-End Testing with Protractor,One of the reasons AngularJS is so great to wo...,HTML,en,USER_0256,NaN,NaN
3,ARTICLE_0003,Corporate venture growth in Brazil is another ...,Despite recent positive news and a renewed int...,HTML,en,USER_1304,NaN,NaN
4,ARTICLE_0004,Cross-channel user experiences with Drupal (aw...,"Last year around this time, I wrote that The B...",HTML,en,USER_0336,NaN,NaN
...,...,...,...,...,...,...,...,...
3003,ARTICLE_3003,Como consumir conteúdo de qualidade em iOS - C...,"Quando iniciei minha jornada em Swift, saindo ...",HTML,pt,USER_0882,BR,MG
3004,ARTICLE_3004,Aurelia 1.0 is Here!!!,It's been an amazing journey to get here and i...,HTML,en,USER_0220,NaN,NaN
3005,ARTICLE_3005,Lessons from converting an app to 100% Kotlin ...,This is part one in a series of posts about Ko...,HTML,en,USER_1010,BR,SP
3006,ARTICLE_3006,ITA está oferecendo 10 cursos gratuitos a dist...,"O Instituto Tecnológico de Aeronáutica (ITA) ,...",HTML,pt,USER_1210,NaN,NaN


In [19]:
article_info[article_info['Format'] != 'HTML']['articleID']

17      ARTICLE_0017
67      ARTICLE_0067
209     ARTICLE_0209
288     ARTICLE_0288
531     ARTICLE_0531
761     ARTICLE_0761
793     ARTICLE_0793
1100    ARTICLE_1100
1105    ARTICLE_1105
1199    ARTICLE_1199
1256    ARTICLE_1256
1631    ARTICLE_1631
1688    ARTICLE_1688
1878    ARTICLE_1878
1975    ARTICLE_1975
2014    ARTICLE_2014
2249    ARTICLE_2249
2853    ARTICLE_2853
2860    ARTICLE_2860
2988    ARTICLE_2988
Name: articleID, dtype: object

In [11]:
submission

,userID,articleID
0,USER_0000,ARTICLE_0000
1,USER_0000,ARTICLE_0001
2,USER_0000,ARTICLE_0002
3,USER_0000,ARTICLE_0003
4,USER_0000,ARTICLE_0004
...,...,...
7070,USER_1420,ARTICLE_0000
7071,USER_1420,ARTICLE_0001
7072,USER_1420,ARTICLE_0002
7073,USER_1420,ARTICLE_0003


In [5]:
user_article_matrix = view_log_train.groupby(['userID', 'articleID']).size().unstack(fill_value=0)
user_article_matrix

articleID,ARTICLE_0000,ARTICLE_0001,ARTICLE_0002,ARTICLE_0003,ARTICLE_0004,ARTICLE_0005,ARTICLE_0006,ARTICLE_0007,ARTICLE_0008,ARTICLE_0009,...,ARTICLE_2998,ARTICLE_2999,ARTICLE_3000,ARTICLE_3001,ARTICLE_3002,ARTICLE_3003,ARTICLE_3004,ARTICLE_3005,ARTICLE_3006,ARTICLE_3007
userID,,,,,,,,,,,,,,,,,,,,,
USER_0000,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
USER_0001,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
USER_0002,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
USER_0003,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
USER_0004,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
USER_1416,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
USER_1417,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
USER_1418,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
# 사용자-기사 행렬 생성
user_article_matrix = view_log_train.groupby(['userID', 'articleID']).size().unstack(fill_value=0)

# 사용자 간의 유사성 계산
user_similarity = cosine_similarity(user_article_matrix)

# 추천 점수 계산
user_predicted_scores = user_similarity.dot(user_article_matrix) / np.array([np.abs(user_similarity).sum(axis=1)]).T


In [ ]:
# 이미 조회한 기사 포함해서 추천
recommendations = []
for idx, user in enumerate(user_article_matrix.index):    
    # 해당 사용자의 추천 점수 (높은 점수부터 정렬)
    sorted_indices = user_predicted_scores[idx].argsort()[::-1]
    top5recommend = [article for article in user_article_matrix.columns[sorted_indices]][:5]
    
    for article in top5recommend:
        recommendations.append([user, article])
        
# sample_submission.csv 형태로 DataFrame 생성
top_recommendations = pd.DataFrame(recommendations, columns=['userID', 'articleID'])

submission['articleID'] = top_recommendations['articleID']

submission.to_csv('baseline_submission.csv', index=False)
